In [2]:
# we will create a class to connect to sqlite database and perform CRUD operations

# first we need to import sqlite3

import sqlite3
import pandas as pd

In [5]:
# our class for connecting to sqlite database and performing CRUD operations
# especially useful if you have multiple databases and you want to create a class for each database

# later on we can move this class to separate file and import it TODO
class SqliteCRUD:
    
    # constructor
    # called when we create an object of this class
    def __init__(self, dbpath):
        self.dbpath = dbpath
        self.conn = sqlite3.connect(dbpath)
        self.cursor = self.conn.cursor()
        # print status  
        print(f"Connected to database: {dbpath} Ready to perform CRUD operations!")
    
    # destructor - will be called when object is deleted - typically when program ends
    def __del__(self):
        print("Closing connection")
        self.conn.close()
    
    # get a list of all artists
    def read_artists(self):
        self.cursor.execute("SELECT * FROM artists")
        return self.cursor.fetchall() # returns a list of tuples

    def create_artist(self, name):
        # notice the parameterized query - for security reasons
        self.cursor.execute("INSERT INTO artists (name) VALUES (?)", (name,))
        self.conn.commit()
        # return self to allow chaining
        return self

    # insert a list of artists
    def create_artists(self, artists):
        self.cursor.executemany("INSERT INTO artists (name) VALUES (?)", artists)
        # alternative:
        # for artist in artists:
        #     self.cursor.execute("INSERT INTO artists (name) VALUES (?)", artist)
        # should be similar speed as long as we do not have too many artists
        # also note how we only commit once
        self.conn.commit()
        return self

    # update artist name
    def update_artist_by_id(self, id, new_name):
        # again parameterized query for security reasons
        self.cursor.execute("UPDATE artists SET name = ? WHERE ArtistId = ?", (new_name, id))
        self.conn.commit()
        return self

    # update arits name by name
    def update_artist_by_name(self, old_name, new_name):
        # again parameterized query for security reasons
        # this could change multiple rows !
        self.cursor.execute("UPDATE artists SET name = ? WHERE name = ?", (new_name, old_name))
        self.conn.commit()
        return self

    # let's get tail of artists table
    def read_artists_tail(self, n=10):
        self.cursor.execute("SELECT * FROM artists ORDER BY ArtistId DESC LIMIT ?", (n,))
        return self.cursor.fetchall()

    # delete artist by id
    def delete_artist_by_id(self, id):
        # again parameterized query for security reasons
        self.cursor.execute("DELETE FROM artists WHERE ArtistId = ?", (id,))
        self.conn.commit()
        return self

    # delete artist by name
    def delete_artist_by_name(self, name):
        # again parameterized query for security reasons
        self.cursor.execute("DELETE FROM artists WHERE name = ?", (name,))
        self.conn.commit()
        return self

    # return dataframe with all artists
    def read_artists_df(self):
        # by abstracting this we can change the implementation later
        # also we create documentation for this method
        return pd.read_sql_query("SELECT * FROM artists", self.conn)

    def read_albums_df(self):
        return pd.read_sql_query("SELECT * FROM albums", self.conn)

    def read_tracks_df(self):
        return pd.read_sql_query("SELECT * FROM tracks", self.conn)

    def read_album_stats(self):
        query = """
        --how about getting total running length of each album?
        SELECT
            t.albumid,
            a.Title AlbumTitle,
            a2.Name ArtistName,
            COUNT(trackid),
            SUM(Milliseconds)/(1000*60) AlbumLengthMinutes,
            AVG(Milliseconds)/(1000*60) AverageSongLengthMin,
            MIN(Milliseconds)/(1000*60) ShortestSongMin,
            MAX(Milliseconds)/(1000*60) LongestSongMin
        FROM
            tracks t
        JOIN albums a 
        ON a.AlbumId = t.AlbumId 
        JOIN artists a2 
        ON a.ArtistId = a2.ArtistId 
        GROUP BY
            t.albumid
        --SO HAVING is similar to WHERE but we use it in GROUP BY situations
        -- disabling HAVING it for DF since we can filter there 
        -- HAVING AlbumLengthMinutes < 80
        ORDER BY AlbumLengthMinutes DESC;
        """
        return pd.read_sql_query(query, self.conn)

    # just add more methods for other tables, joins, views, etc.

    # TODO explore GROUP BY, HAVING and other SQL features

In [39]:
# in effect we have made our own ORM - object relational mapper
# ORM is a library that maps database tables to python objects

# ORM on wikipedia: https://en.wikipedia.org/wiki/Object-relational_mapping

In [6]:
# we create a single object of this class
db = SqliteCRUD("chinook.db")
db.dbpath
# now we can run queries using db.cursor.execute(query) method

Connected to database: chinook.db Ready to perform CRUD operations!


'chinook.db'

In [4]:
# now let's close the connection by deleting the object
del db

Closing connection


In [9]:
artist_list = db.read_artists()
artist_list[:3]

[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith')]

In [10]:
# last 3
artist_list[-3:]

[(278, 'Jumprava'),
 (279, '; DROP TABLE artists; --'),
 (280, 'Lady Gaga; DROP TABLES artists; --')]

In [11]:
# create a new artist
db.create_artist("David Bowie")
artist_list = db.read_artists()
# last 3 
artist_list[-3:]


[(279, '; DROP TABLE artists; --'),
 (280, 'Lady Gaga; DROP TABLES artists; --'),
 (281, 'David Bowie')]

In [12]:
# a list of single element tuples
my_artists = [("David Bowie",), ("Queen",), ("The Beatles",)]
db.create_artists(my_artists)
artist_list = db.read_artists()
# last 3
artist_list[-3:]

[(282, 'David Bowie'), (283, 'Queen'), (284, 'The Beatles')]

In [17]:
# last 10 artists
artist_list = db.read_artists()
artist_list[-10:]

[(275, 'Philip Glass Ensemble'),
 (276, 'Valdis Saulespurēns'),
 (277, 'Jumprava'),
 (278, 'Jumprava'),
 (279, '; DROP TABLE artists; --'),
 (280, 'Lady Gaga; DROP TABLES artists; --'),
 (281, 'David Bowie'),
 (282, 'David Bowie'),
 (283, 'Queen'),
 (284, 'The Beatles')]

In [21]:
# let's change 280 to Lady Gaga
db.update_artist_by_id(280, "Lady Gaga")
db.read_artists()[-10:] # of course on larger database we would not read all artists

[(275, 'Philip Glass Ensemble'),
 (276, 'Valdis Saulespurēns'),
 (277, 'Jumprava'),
 (278, 'Jumprava'),
 (279, '; DROP TABLE artists; --'),
 (280, 'Lady Gaga'),
 (281, 'David Bowie'),
 (282, 'David Bowie'),
 (283, 'Queen'),
 (284, 'The Beatles')]

In [25]:
# let's get tail
db.read_artists_tail(12) # default is 10

[(284, 'The Beatles'),
 (283, 'Queen'),
 (282, 'David Bowie'),
 (281, 'David Bowie'),
 (280, 'Lady Gaga'),
 (279, '; DROP TABLE artists; --'),
 (278, 'Jumprava'),
 (277, 'Jumprava'),
 (276, 'Valdis Saulespurēns'),
 (275, 'Philip Glass Ensemble'),
 (274, 'Nash Ensemble'),
 (273,
  'C. Monteverdi, Nigel Rogers - Chiaroscuro; London Baroque; London Cornett & Sackbu')]

In [28]:
# let's get tail of 5
db.read_artists_tail(5)

[(284, 'The Beatles'),
 (283, 'Queen'),
 (282, 'David Bowie'),
 (281, 'David Bowie'),
 (280, 'Lady Gaga')]

In [29]:
# let's chain some methods i am using \ for newline to make it more readable
db.create_artist("Ansis")\
    .create_artist("Lauris Reinikis")\
    .create_artist("Samanta Tīne")\
    .read_artists_tail(8)

[(287, 'Samanta Tīne'),
 (286, 'Lauris Reinikis'),
 (285, 'Ansis'),
 (284, 'The Beatles'),
 (283, 'Queen'),
 (282, 'David Bowie'),
 (281, 'David Bowie'),
 (280, 'Lady Gaga')]

In [34]:
artists_df = db.read_artists_df()
artists_df.head()

ArtistId               Name
0         1              AC/DC
1         2             Accept
2         3          Aerosmith
3         4  Alanis Morissette
4         5    Alice In Chains

In [35]:
artists_df.tail()

ArtistId             Name
282       283            Queen
283       284      The Beatles
284       285            Ansis
285       286  Lauris Reinikis
286       287     Samanta Tīne

In [ ]:
# now we could start working on the dataframe without worrying about the database

In [7]:
# get album stats dataframe
album_stats_df = db.read_album_stats()
album_stats_df.head()

AlbumId                                AlbumTitle  \
0      229                            Lost, Season 3   
1      253  Battlestar Galactica (Classic), Season 1   
2      230                            Lost, Season 1   
3      231                            Lost, Season 2   
4      228                          Heroes, Season 1   

                       ArtistName  COUNT(trackid)  AlbumLengthMinutes  \
0                            Lost              26                1177   
1  Battlestar Galactica (Classic)              24                1170   
2                            Lost              25                1080   
3                            Lost              24                1054   
4                          Heroes              23                 996   

   AverageSongLengthMin  ShortestSongMin  LongestSongMin  
0             45.298450               42              84  
1             48.759572               47              49  
2             43.236624               40              43  
3             43.951133               41              47  
4             43.319035               42              44

In [8]:
class GROUPBYDB(SqliteCRUD):
   # no need for constructor and destructor
   # because we are inheriting them from parent class 
# we can add more methods here
   
    def total_sales_by_year(self):
        query = """
        SELECT SUBSTRING(InvoiceDate,1,4) as year,sum(Total) as total_sales
        from invoices i 
        group by SUBSTRING(InvoiceDate,1,4)
        """
        return pd.read_sql_query(query, self.conn)

    def total_sales_by_country(self):
        query = """
        SELECT c.Country ,sum(Total) as total_sales
        from invoices i 
        join customers c on i.CustomerId =c.CustomerId 
        group by c.Country 
        """
        return pd.read_sql_query(query, self.conn)

    def count_track(self): 
        query="""
        SELECT p.Name , count(pt.TrackId) as count_track
        from playlist_track pt 
        join playlists p on pt.PlaylistId =p.PlaylistId 
        group by pt.PlaylistId
        """
        return pd.read_sql_query(query,self.conn)
    
    def track_length(self): 
        query="""
        SELECT p.Name , count(pt.TrackId) as count_track, sum(t.Milliseconds)  / (1000 * 60)  as length_minutes
        from playlist_track pt 
        join playlists p on pt.PlaylistId =p.PlaylistId 
        join tracks t on pt.TrackId =t.TrackId 
        group by pt.PlaylistId
        """
        return pd.read_sql_query(query,self.conn)

In [9]:
del db

Closing connection


In [10]:
db = GROUPBYDB("chinook.db") # it will use constructor from parent class

Connected to database: chinook.db Ready to perform CRUD operations!


In [11]:
db.total_sales_by_year()

year  total_sales
0  2009       449.46
1  2010       481.45
2  2011       469.58
3  2012       477.53
4  2013       450.58

In [13]:
# save the sales report to csv
# rounding 2 places
db.total_sales_by_year().round(2).to_csv("sales_by_year.csv", index=False)
# round 2 is pandas method to round to 2 decimal places

In [14]:
# read playlist track
db.track_length()

Name  count_track  length_minutes
0                        Music         3290           14628
1                     TV Shows          213            8351
2                   90’s Music         1477            6645
3                        Music         3290           14628
4                 Music Videos            1               4
5                     TV Shows          213            8351
6              Brazilian Music           39             158
7                    Classical           75             362
8    Classical 101 - Deep Cuts           25             112
9   Classical 101 - Next Steps           25             126
10  Classical 101 - The Basics           25             123
11                      Grunge           15              68
12         Heavy Metal Classic           26             136
13                 On-The-Go 1            1               3